#### Install Required Packages

In [1]:
!pip install sqlalchemy psycopg2-binary

In [7]:
import pandas as pd
from sqlalchemy import create_engine
from sqlalchemy import text

# 1. Data Preparation

##  Load CSVs with Pandas

In [13]:
# Load the CSV files
providers_df = pd.read_csv("providers_data.csv")
receivers_df = pd.read_csv("receivers_data.csv")
food_df = pd.read_csv("food_listings_data.csv", parse_dates=["Expiry_Date"])
claims_df = pd.read_csv("claims_data.csv", parse_dates=["Timestamp"])

In [31]:
# Preview the providers datasets
print(providers_df.head())

   Provider_ID                         Name           Type  \
0            1             Gonzales-Cochran    Supermarket   
1            2  Nielsen, Johnson and Fuller  Grocery Store   
2            3                 Miller-Black    Supermarket   
3            4   Clark, Prince and Williams  Grocery Store   
4            5               Coleman-Farley  Grocery Store   

                                             Address            City  \
0  74347 Christopher Extensions\nAndreamouth, OK ...     New Jessica   
1           91228 Hanson Stream\nWelchtown, OR 27136     East Sheena   
2  561 Martinez Point Suite 507\nGuzmanchester, W...  Lake Jesusview   
3     467 Bell Trail Suite 409\nPort Jesus, IA 61188     Mendezmouth   
4  078 Matthew Creek Apt. 319\nSaraborough, MA 53978   Valentineside   

                Contact  
0       +1-600-220-0480  
1  +1-925-283-8901x6297  
2      001-517-295-2206  
3      556.944.8935x401  
4          193.714.6577  


In [29]:
# Preview the receivers datasets
print(receivers_df.head())

   Receiver_ID          Name        Type               City           Contact
0            1  Donald Gomez     Shelter     Port Carlburgh     (955)922-5295
1            2  Laurie Ramos  Individual         Lewisburgh      761.042.1570
2            3  Ashley Mckee         NGO  South Randalltown  691-023-0094x856
3            4    Erika Rose         NGO   South Shaneville        8296491111
4            5   John Romero  Individual          Bakerport      067.491.0154


In [33]:
# Preview the food_listings datasets
print(food_df.head())

   Food_ID Food_Name  Quantity Expiry_Date  Provider_ID     Provider_Type  \
0        1     Bread        43  2025-03-17          110     Grocery Store   
1        2      Soup        22  2025-03-24          791     Grocery Store   
2        3    Fruits        46  2025-03-28          478  Catering Service   
3        4    Fruits        15  2025-03-16          930        Restaurant   
4        5      Soup        14  2025-03-19          279        Restaurant   

           Location       Food_Type  Meal_Type  
0  South Kellyville  Non-Vegetarian  Breakfast  
1        West James  Non-Vegetarian     Dinner  
2       Lake Regina           Vegan  Breakfast  
3         Kellytown           Vegan      Lunch  
4        Garciaport           Vegan     Dinner  


In [35]:
# Preview the claims datasets
print(claims_df.head())

   Claim_ID  Food_ID  Receiver_ID     Status           Timestamp
0         1      164          908    Pending 2025-03-05 05:26:00
1         2      353          391  Cancelled 2025-03-11 10:24:00
2         3      626          492  Completed 2025-03-21 00:59:00
3         4       61          933  Cancelled 2025-03-04 09:08:00
4         5      345          229    Pending 2025-03-14 15:17:00


In [37]:
providers_df

,Provider_ID,Name,Type,Address,City,Contact
0,1,Gonzales-Cochran,Supermarket,"74347 Christopher Extensions\nAndreamouth, OK ...",New Jessica,+1-600-220-0480
1,2,"Nielsen, Johnson and Fuller",Grocery Store,"91228 Hanson Stream\nWelchtown, OR 27136",East Sheena,+1-925-283-8901x6297
2,3,Miller-Black,Supermarket,"561 Martinez Point Suite 507\nGuzmanchester, W...",Lake Jesusview,001-517-295-2206
3,4,"Clark, Prince and Williams",Grocery Store,"467 Bell Trail Suite 409\nPort Jesus, IA 61188",Mendezmouth,556.944.8935x401
4,5,Coleman-Farley,Grocery Store,"078 Matthew Creek Apt. 319\nSaraborough, MA 53978",Valentineside,193.714.6577
...,...,...,...,...,...,...
995,996,"Vasquez, Ruiz and Flowers",Restaurant,"84308 Justin Stravenue\nNew Amberside, NE 53447",Williamview,+1-319-378-7627x0682
996,997,Garza-Williams,Catering Service,"08864 Figueroa Radial Suite 948\nJennaberg, AZ...",East Rossside,001-924-441-3963x746
997,998,Novak Group,Grocery Store,"934 Zachary Run\nMelissamouth, WY 02729",Joshuastad,(903)642-1969x3300
998,999,Moody Ltd,Grocery Store,"17580 Ernest Hills\nLake Michaelmouth, OR 56416",Stevenchester,637.300.3664x4880


In [39]:
receivers_df

,Receiver_ID,Name,Type,City,Contact
0,1,Donald Gomez,Shelter,Port Carlburgh,(955)922-5295
1,2,Laurie Ramos,Individual,Lewisburgh,761.042.1570
2,3,Ashley Mckee,NGO,South Randalltown,691-023-0094x856
3,4,Erika Rose,NGO,South Shaneville,8296491111
4,5,John Romero,Individual,Bakerport,067.491.0154
...,...,...,...,...,...
995,996,Matthew Curtis,Shelter,Lanechester,127-889-4442x1289
996,997,Amanda Cain,NGO,New Steven,+1-001-491-5601x5316
997,998,Theodore Briggs,Individual,South Sandra,930-609-9442x5031
998,999,Cheyenne Ramsey,NGO,Lake Jeffery,001-326-320-4816x15300


In [41]:
food_df

,Food_ID,Food_Name,Quantity,Expiry_Date,Provider_ID,Provider_Type,Location,Food_Type,Meal_Type
0,1,Bread,43,2025-03-17,110,Grocery Store,South Kellyville,Non-Vegetarian,Breakfast
1,2,Soup,22,2025-03-24,791,Grocery Store,West James,Non-Vegetarian,Dinner
2,3,Fruits,46,2025-03-28,478,Catering Service,Lake Regina,Vegan,Breakfast
3,4,Fruits,15,2025-03-16,930,Restaurant,Kellytown,Vegan,Lunch
4,5,Soup,14,2025-03-19,279,Restaurant,Garciaport,Vegan,Dinner
...,...,...,...,...,...,...,...,...,...
995,996,Fish,15,2025-03-30,467,Catering Service,Phillipsfort,Vegan,Breakfast
996,997,Fish,22,2025-03-18,35,Grocery Store,Andersonmouth,Vegetarian,Breakfast
997,998,Fruits,6,2025-03-22,444,Restaurant,New Billy,Non-Vegetarian,Dinner
998,999,Pasta,15,2025-03-30,702,Supermarket,Lake Mistyton,Non-Vegetarian,Lunch


In [43]:
claims_df

,Claim_ID,Food_ID,Receiver_ID,Status,Timestamp
0,1,164,908,Pending,2025-03-05 05:26:00
1,2,353,391,Cancelled,2025-03-11 10:24:00
2,3,626,492,Completed,2025-03-21 00:59:00
3,4,61,933,Cancelled,2025-03-04 09:08:00
4,5,345,229,Pending,2025-03-14 15:17:00
...,...,...,...,...,...
995,996,855,211,Completed,2025-03-13 19:40:00
996,997,980,746,Cancelled,2025-03-17 22:04:00
997,998,832,967,Cancelled,2025-03-13 18:00:00
998,999,917,90,Completed,2025-03-01 15:31:00


## Clean & Format Data Types

In [47]:
# Convert IDs and numeric columns to integers
providers_df["Provider_ID"] = providers_df["Provider_ID"].astype(int)
receivers_df["Receiver_ID"] = receivers_df["Receiver_ID"].astype(int)
food_df["Food_ID"] = food_df["Food_ID"].astype(int)
food_df["Quantity"] = food_df["Quantity"].astype(int)
food_df["Provider_ID"] = food_df["Provider_ID"].astype(int)
claims_df["Claim_ID"] = claims_df["Claim_ID"].astype(int)
claims_df["Food_ID"] = claims_df["Food_ID"].astype(int)
claims_df["Receiver_ID"] = claims_df["Receiver_ID"].astype(int)

In [49]:
# Ensure string columns are clean
string_cols = [
    "Name", "Type", "Address", "City", "Contact",
    "Food_Name", "Provider_Type", "Location", "Food_Type", "Meal_Type", "Status"
]

for df in [providers_df, receivers_df, food_df, claims_df]:
    for col in df.columns:
        if col in string_cols:
            df[col] = df[col].astype(str).str.strip()

## Validate Relationships

In [52]:
# Validate Food.Provider_ID refers to Providers.Provider_ID
invalid_provider_ids = food_df[~food_df["Provider_ID"].isin(providers_df["Provider_ID"])]
print("Invalid Provider_IDs in food_listings:", invalid_provider_ids)

Invalid Provider_IDs in food_listings: Empty DataFrame
Columns: [Food_ID, Food_Name, Quantity, Expiry_Date, Provider_ID, Provider_Type, Location, Food_Type, Meal_Type]
Index: []


In [54]:
# Validate Claims.Receiver_ID refers to Receivers.Receiver_ID
invalid_receiver_ids = claims_df[~claims_df["Receiver_ID"].isin(receivers_df["Receiver_ID"])]
print("Invalid Receiver_IDs in claims:", invalid_receiver_ids)

Invalid Receiver_IDs in claims: Empty DataFrame
Columns: [Claim_ID, Food_ID, Receiver_ID, Status, Timestamp]
Index: []


In [56]:
# Validate Claims.Food_ID refers to Food.Food_ID
invalid_food_ids = claims_df[~claims_df["Food_ID"].isin(food_df["Food_ID"])]
print("Invalid Food_IDs in claims:", invalid_food_ids)

Invalid Food_IDs in claims: Empty DataFrame
Columns: [Claim_ID, Food_ID, Receiver_ID, Status, Timestamp]
Index: []


## Handle Missing or Null Values

In [59]:
# Check and fill or drop missing values
print(food_df.isnull().sum())
food_df = food_df.dropna()


Food_ID          0
Food_Name        0
Quantity         0
Expiry_Date      0
Provider_ID      0
Provider_Type    0
Location         0
Food_Type        0
Meal_Type        0
dtype: int64


# 2. Database Design and SQL Setup

## Setup Database Connection

In [63]:
# Define PostgreSQL credentials
username = 'postgres'
password = 'guvi'
host = 'localhost'       
port = '5432'             
database = 'food_management_db'

# Create the connection string
connection_string = f'postgresql+psycopg2://{username}:{password}@{host}:{port}/{database}'

# Create the SQLAlchemy engine
engine = create_engine(connection_string)

# Test the connection
try:
    with engine.connect() as connection:
        print("Connected to PostgreSQL database successfully!")
except Exception as e:
    print("Connection failed:", e)


Connected to PostgreSQL database successfully!


## PostgreSQL SQL Schema(Create Tables)

In [66]:
# Create Tables
schema_sql = """
-- Table: providers
CREATE TABLE IF NOT EXISTS providers (
    Provider_ID INTEGER PRIMARY KEY,
    Name TEXT NOT NULL,
    Type TEXT,
    Address TEXT,
    City TEXT,
    Contact TEXT
);

-- Table: receivers
CREATE TABLE IF NOT EXISTS receivers (
    Receiver_ID INTEGER PRIMARY KEY,
    Name TEXT NOT NULL,
    Type TEXT,
    City TEXT,
    Contact TEXT
);

-- Table: food_listings
CREATE TABLE IF NOT EXISTS food_listings (
    Food_ID INTEGER PRIMARY KEY,
    Food_Name TEXT,
    Quantity INTEGER,
    Expiry_Date DATE,
    Provider_ID INTEGER REFERENCES providers(Provider_ID) ON DELETE CASCADE,
    Provider_Type TEXT,
    Location TEXT,
    Food_Type TEXT,
    Meal_Type TEXT
);

-- Table: claims
CREATE TABLE IF NOT EXISTS claims (
    Claim_ID INTEGER PRIMARY KEY,
    Food_ID INTEGER REFERENCES food_listings(Food_ID) ON DELETE CASCADE,
    Receiver_ID INTEGER REFERENCES receivers(Receiver_ID) ON DELETE CASCADE,
    Status TEXT,
    Timestamp TIMESTAMP
);
"""

# Execute the SQL
with engine.connect() as conn:
    conn.execute(text(schema_sql))
    print("Tables created successfully!")


Tables created successfully!


## Python Script to Insert Data into PostgreSQL

In [73]:
# Insert data into each table
providers_df.to_sql("providers", engine, if_exists="append", index=False)
receivers_df.to_sql("receivers", engine, if_exists="append", index=False)
food_df.to_sql("food_listings", engine, if_exists="append", index=False)
claims_df.to_sql("claims", engine, if_exists="append", index=False)

print("Data inserted into PostgreSQL successfully.")


Data inserted into PostgreSQL successfully.


In [75]:
# Query a few rows from providers table
with engine.connect() as conn:
    result = conn.execute(text("SELECT * FROM providers LIMIT 5"))
    for row in result:
        print(row)

(1, 'Gonzales-Cochran', 'Supermarket', '74347 Christopher Extensions\nAndreamouth, OK 91839', 'New Jessica', '+1-600-220-0480')
(2, 'Nielsen, Johnson and Fuller', 'Grocery Store', '91228 Hanson Stream\nWelchtown, OR 27136', 'East Sheena', '+1-925-283-8901x6297')
(3, 'Miller-Black', 'Supermarket', '561 Martinez Point Suite 507\nGuzmanchester, WA 94320', 'Lake Jesusview', '001-517-295-2206')
(4, 'Clark, Prince and Williams', 'Grocery Store', '467 Bell Trail Suite 409\nPort Jesus, IA 61188', 'Mendezmouth', '556.944.8935x401')
(5, 'Coleman-Farley', 'Grocery Store', '078 Matthew Creek Apt. 319\nSaraborough, MA 53978', 'Valentineside', '193.714.6577')


In [77]:
# Query a few rows from receivers table
with engine.connect() as conn:
    result = conn.execute(text("SELECT * FROM receivers LIMIT 5"))
    for row in result:
        print(row)

(1, 'Donald Gomez', 'Shelter', 'Port Carlburgh', '(955)922-5295')
(2, 'Laurie Ramos', 'Individual', 'Lewisburgh', '761.042.1570')
(3, 'Ashley Mckee', 'NGO', 'South Randalltown', '691-023-0094x856')
(4, 'Erika Rose', 'NGO', 'South Shaneville', '8296491111')
(5, 'John Romero', 'Individual', 'Bakerport', '067.491.0154')


In [79]:
# Query a few rows from food-listings table
with engine.connect() as conn:
    result = conn.execute(text("SELECT * FROM food_listings LIMIT 5"))
    for row in result:
        print(row)

(1, 'Bread', 43, datetime.datetime(2025, 3, 17, 0, 0), 110, 'Grocery Store', 'South Kellyville', 'Non-Vegetarian', 'Breakfast')
(2, 'Soup', 22, datetime.datetime(2025, 3, 24, 0, 0), 791, 'Grocery Store', 'West James', 'Non-Vegetarian', 'Dinner')
(3, 'Fruits', 46, datetime.datetime(2025, 3, 28, 0, 0), 478, 'Catering Service', 'Lake Regina', 'Vegan', 'Breakfast')
(4, 'Fruits', 15, datetime.datetime(2025, 3, 16, 0, 0), 930, 'Restaurant', 'Kellytown', 'Vegan', 'Lunch')
(5, 'Soup', 14, datetime.datetime(2025, 3, 19, 0, 0), 279, 'Restaurant', 'Garciaport', 'Vegan', 'Dinner')


In [81]:
# Query a few rows from claims table
with engine.connect() as conn:
    result = conn.execute(text("SELECT * FROM claims LIMIT 5"))
    for row in result:
        print(row)

(1, 164, 908, 'Pending', datetime.datetime(2025, 3, 5, 5, 26))
(2, 353, 391, 'Cancelled', datetime.datetime(2025, 3, 11, 10, 24))
(3, 626, 492, 'Completed', datetime.datetime(2025, 3, 21, 0, 59))
(4, 61, 933, 'Cancelled', datetime.datetime(2025, 3, 4, 9, 8))
(5, 345, 229, 'Pending', datetime.datetime(2025, 3, 14, 15, 17))


## CRUD Operations

### Providers 

#### Create(Insert)

In [92]:
def insert_provider(engine, data):
    query = """
    INSERT INTO providers ("Provider_ID", "Name", "Type", "Address", "City", "Contact")
    VALUES (:Provider_ID, :Name, :Type, :Address, :City, :Contact)
    """
    with engine.connect() as conn:
        conn.execute(text(query), data)
        conn.commit()

In [94]:
# Insert a new provider
insert_provider(engine, {
    "Provider_ID": 101,
    "Name": "Good Kitchen",
    "Type": "NGO",
    "Address": "45 Street Name",
    "City": "Hyderabad",
    "Contact": "9876543210"
})

In [ ]:
def update_provider(engine, provider_id, new_city):
    query = "UPDATE providers SET City = :City WHERE Provider_ID = :Provider_ID"
    with engine.connect() as conn:
        conn.execute(text(query), {"City": new_city, "Provider_ID": provider_id})
        conn.commit()

def delete_provider(engine, provider_id):
    query = "DELETE FROM providers WHERE Provider_ID = :Provider_ID"
    with engine.connect() as conn:
        conn.execute(text(query), {"Provider_ID": provider_id})
        conn.commit()

def read_providers(engine):
    query = "SELECT * FROM providers"
    with engine.connect() as conn:
        result = conn.execute(text(query)).fetchall()
        return result


In [ ]:
# Read all providers
providers = read_providers(engine)
for p in providers:
    print(p)

# Update city for a provider
update_provider(engine, 101, "Bangalore")

# Delete a provider
delete_provider(engine, 101)